In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import random


from random import choices
from scipy import stats
from tqdm.notebook import tqdm

In [ ]:
from jupyter_utils import mean_std, display_test, display_group_test, scatter_annotate, show_corrtest_mask_corr
from ortogonolize_utils import draw_scatter, draw_corrected_scatter
from ortogonolize_utils import compute_coefficient, compute_ortogonolized_coefficient, compute_ortogonolized_logit

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings(action='ignore', message='All-NaN slice encountered')
warnings.filterwarnings(action='ignore', message='Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.')
warnings.filterwarnings(action='ignore', message='Mean of empty slice')

In [ ]:
PATH = '/Users/galina.ryazanskaya/Downloads/thesis?/code?/transcripts_NET/'

# Patients

In [ ]:
df = pd.read_excel(PATH +'NAP/GR_all6M_MPP.xlsx', index_col=0, na_values=[' ', '', -99.0])
df = df[df.index.notnull()].drop(columns=['Unnamed: 98', 'Unnamed: 99'])

In [ ]:
# total positve ftd - saps34; total alogia - sans12
# sex_nr - female = 1, male = 0
# ausja_bl = education years at baseline (i.e. 6 months before the NET interview)
# pa - panss

# df.columns

In [ ]:
# print(*enumerate('PANSS P1	PANSS P2	PANSS P3	PANSS P4	PANSS P5	PANSS P6	PANSS P7	PANSS N1	PANSS N2	PANSS N3	PANSS N4	PANSS N5	PANSS N6	PANSS N7	PANSS O1	PANSS O2	PANSS O3	PANSS O4	PANSS O5	PANSS O6	PANSS O7	PANSS O8	PANSS 09	PANSS O10	PANSS O11	PANSS O12	PANSS O13	PANSS O14	PANSS O15	PANSS 016'.split('\t')))

In [ ]:
df['panss_pos'] = df[['pa_01', 'pa_02', 'pa_03', 'pa_04', 'pa_05', 'pa_06', 'pa_07']].sum(axis=1)
df['panss_neg'] = df[['pa_08', 'pa_09', 'pa_10', 'pa_11', 'pa_12', 'pa_13', 'pa_14', 'pa_15']].sum(axis=1)
df['panss_o'] = df[['pa_16', 'pa_17', 'pa_18', 'pa_19', 'pa_20', 'pa_21', 'pa_22', 'pa_23', 'pa_24', 'pa_25', 'pa_26', 'pa_27', 'pa_28', 'pa_29', 'pa_30']].sum(axis=1)
df['panss_total'] = df[['panss_pos', 'panss_neg', 'panss_o']].sum(axis=1)
df['saps_total'] = df[['saps1', 'saps2', 'saps3', 'saps4', 'saps5', 'saps6', 'saps7', 'saps8', 'saps9', 'saps10', 'saps11', 'saps12', 'saps13', 'saps14', 'saps15', 'saps16', 'saps17', 'saps18', 'saps19', 'saps20', 'saps21', 'saps22', 'saps23', 'saps24', 'saps25', 'saps26', 'saps27', 'saps28', 'saps29', 'saps30', 'saps31', 'saps32', 'saps33', 'saps34', 'saps35']].sum(axis=1)
df['sans_total'] =  df[['sans1', 'sans2', 'sans3', 'sans4', 'sans5', 'sans6', 'sans7', 'sans8', 'sans9', 'sans10', 'sans11', 'sans12', 'sans13', 'sans14', 'sans15', 'sans16', 'sans17', 'sans18', 'sans19', 'sans20', 'sans21', 'sans22', 'sans23', 'sans24']].sum(axis=1)

#### Filter out only the patients there are transcripts for

In [ ]:
# df.index

In [ ]:
df.index.rename('ID', inplace=True)

In [ ]:
res = pd.read_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/processed_values/new/de_patients.tsv', sep='\t', index_col=0, header=[0, 1, 2])

In [ ]:
res.dropna(how='all', inplace=True)

In [ ]:
idxs = sorted([x.replace('_6M_manual_c', '') for x in res.index])
idxs = ['MPP_0' + i if len(i) > 1 else 'MPP_00' + i for i in idxs]

In [ ]:
res.index = ['MPP_0' + i if len(i) > 1 else 'MPP_00' + i for i in [x.replace('_6M_manual_c', '') for x in res.index]]
res.index.rename('ID', inplace=True)

In [ ]:
set(idxs).difference(set(df.index))

In [ ]:
df = df.loc[idxs]

#### Look at descriptitve stats

In [ ]:
cols = ['age', 'sex_nr', 'Bildungsjahre', 'IQ_kristallin', 'saps34', 'saps_total', 'sans_total', 'sans12', 'panss_pos', 'panss_neg', 'panss_o', 'panss_total']

In [ ]:
df[cols].count()

In [ ]:
mean_std(df, cols)

In [ ]:
mean_std(df, cols, 'sex_nr')

In [ ]:
df.groupby('sex_nr').count()

In [ ]:
df.age.hist();

# Healthy controls

In [ ]:
df_hc = pd.read_csv(PATH + 'HC/NET_HC.csv', index_col=0, na_values=[' ', ''])

In [ ]:
df_hc.columns

#### Look up descriptive stats

In [ ]:
df_hc.count()

In [ ]:
mean_std(df_hc, ['age', 'Bildungsjahre'])

In [ ]:
mean_std(df_hc, ['age', 'Bildungsjahre'], 'sex_nr')

In [ ]:
df_hc.groupby('sex_nr').count()

In [ ]:
# df_hc.index

#### Check that all the transcripts are availble

In [ ]:
res_hc =  pd.read_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/processed_values/new/de_HC.tsv', sep='\t', index_col=0, header=[0, 1, 2])

In [ ]:
idxs_HC = sorted([x.replace('_c', '').replace('NET_', '')  for x in res_hc.index])

In [ ]:
set(idxs_HC).difference(set(df_hc.index))

In [ ]:
df_hc = df_hc.loc[idxs_HC]

In [ ]:
df_hc.index.rename('ID', inplace=True)

In [ ]:
res_hc.index = [x.replace('_c', '').replace('NET_', '')  for x in res_hc.index]
res_hc.index.rename('ID', inplace=True)

In [ ]:
merged_hc = df_hc.merge(res_hc['averaged'], on='ID', how='outer')
corr_hc = merged_hc.corr()

#### Fill NA in syntactic (percentage of unused POS should be zero, not NA)

In [ ]:
cols_LM = [col for col in merged_hc if col[0] == 'LM']
cols_synt = [col for col in merged_hc if col[0] == 'syntactic']
cols_lex = [col for col in merged_hc if col[0] == 'lexical']
cols_graph = [col for col in merged_hc if col[0] == 'graph']

In [ ]:
pos = set([x[1] for x in cols_synt if x[1].isupper()])

In [ ]:
pos.difference(set(('ADJ', 'ADV', 'AUX', 'CCONJ', 'DET','NOUN', 'PRON', 'PROPN', 'SCONJ', 'VERB', 'PART')))

In [ ]:
POS_to_use = ('ADJ', 'ADV', 'AUX', 'CCONJ', 'DET','NOUN', 'PRON', 'PROPN', 'SCONJ', 'VERB', 'PART')

In [ ]:
s = merged_hc[cols_synt].isna().any()
s[s].index

In [ ]:
# merged_hc[s[s].index][merged_hc[cols_synt].isna().any(axis=1)]

In [ ]:
merged_hc[s[s].index] = merged_hc[s[s].index].fillna(0.0)

In [ ]:
merged_hc.drop(columns=[c for c in cols_synt if c[1].isupper() and c[1] not in POS_to_use], inplace=True)

In [ ]:
cols_synt = [col for col in merged_hc if col[0] == 'syntactic']
cols_av = cols_synt + cols_LM + cols_lex + cols_graph

### Analyze the effects of age in healthy controls

In [ ]:
df_hc.age.hist();

In [ ]:
df_hc[df_hc.age > 60]

In [ ]:
corr_hc[abs(corr_hc['age']) > 0.3]['age']

In [ ]:
corr_hc[abs(corr_hc['age']) > 0.3].index

In [ ]:
merged_hc.plot.scatter('age', ('syntactic', 'PRON'));

In [ ]:
merged_hc.plot.scatter('age', ('syntactic', 'AUX'));

In [ ]:
merged_hc.plot.scatter('age', ('graph', 'degree_std'));

In [ ]:
merged_hc.plot.scatter('age', ('syntactic', 'VERB'));

In [ ]:
corr_hc[abs(corr_hc['age']) > 0.5].index

### Analyze the effects of education years in healthy controls

In [ ]:
corr_hc[abs(corr_hc['Bildungsjahre']) > 0.3].index

In [ ]:
corr_hc[abs(corr_hc['Bildungsjahre']) > 0.5].index

In [ ]:
merged_hc.plot.scatter('Bildungsjahre', ('LM', 'm_scoh'));

In [ ]:
stats.pearsonr(corr_hc['Bildungsjahre'], corr_hc[('syntactic', 'std_sent_len')])

In [ ]:
merged_hc.plot.scatter('Bildungsjahre', ('syntactic', 'std_sent_len'));

### Analyze the effects of sentence length in healthy controls

In [ ]:
merged_hc[('syntactic', 'mean_sent_len')].plot.hist();

In [ ]:
corr_hc[abs(corr_hc[('syntactic', 'mean_sent_len')]) > 0.3].index

In [ ]:
corr_hc[abs(corr_hc[('syntactic', 'mean_sent_len')]) > 0.5].index

In [ ]:
corr_hc[abs(corr_hc[('syntactic', 'mean_sent_len')]) > 0.6].index

In [ ]:
corr_hc.loc[corr_hc[abs(corr_hc[('syntactic', 'mean_sent_len')]) > 0.6].index][('syntactic', 'mean_sent_len')].sort_values(key=lambda x: abs(x), ascending=False)

In [ ]:
merged_hc[merged_hc[('syntactic', 'mean_sent_len')] < 24].plot.scatter(('syntactic', 'mean_sent_len'), ('syntactic', 'AUX'));

In [ ]:
merged_hc[merged_hc[('syntactic', 'mean_sent_len')] < 24].plot.scatter(('syntactic', 'mean_sent_len'), ('LM', 'm_scoh'));

In [ ]:
merged_hc[merged_hc[('syntactic', 'mean_sent_len')] < 24].plot.scatter(('syntactic', 'mean_sent_len'), ('LM', 'm_cgcoh'));

In [ ]:
merged_hc[merged_hc[('syntactic', 'mean_sent_len')] < 24].plot.scatter(('syntactic', 'mean_sent_len'), ('LM', 'm_gcoh'));

In [ ]:
merged_hc[merged_hc[('syntactic', 'mean_sent_len')] < 24].plot.scatter(('syntactic', 'mean_sent_len'), ('LM', 'm_lcoh'));

# Merge control and patient data

In [ ]:
df['group'] = 1
df_hc['group'] = 0
compare_groups = pd.concat([df[cols + ['group']], df_hc], axis=0)

In [ ]:
compare_groups.groupby(['group']).count()

In [ ]:
mean_std(compare_groups, ['age', 'Bildungsjahre'], 'group')


In [ ]:
sns.histplot(compare_groups['saps_total']);

In [ ]:
sns.histplot(compare_groups['sans_total']);

In [ ]:
stats.ttest_ind(df['age'], df_hc['age']).pvalue

In [ ]:
stats.ttest_ind(df['Bildungsjahre'], df_hc['Bildungsjahre'], nan_policy='omit').pvalue

In [ ]:
combined_data_averaged = compare_groups.merge(pd.concat([res['averaged'], res_hc['averaged']], axis=0), on='ID', how='outer')

### Fill NA in syntactic  (percentage of unused POS should be zero, not NA)

In [ ]:
s = combined_data_averaged[cols_synt].isna().any()
s[s].index

In [ ]:
# combined_data_averaged[s[s].index][combined_data_averaged[cols_synt].isna().any(axis=1)]

In [ ]:
combined_data_averaged[s[s].index] = combined_data_averaged[s[s].index].fillna(0.0)

In [ ]:
# combined_data_averaged.columns

# Analyze the differences in control factors

In [ ]:
clinical_cols = ['age',
 'Bildungsjahre',
 'IQ_kristallin',
 'saps34',
 'saps_total',
 'sans_total',
 'sans12',
 'panss_pos',
 'panss_neg',
 'panss_o',
 'panss_total']

In [ ]:
# cols_av = [('LM', 'm_lcoh'), ('LM', 'm_gcoh'), ('LM', 'm_cgcoh'), ('LM', 'm_scoh'), ('LM', 'm_sporb'), ('LM', 'm_bert_lcoh'), ('LM', 'm_bert_gcoh'), ('LM', 'm_bert_cgcoh'), ('LM', 'm_bert_scoh'),
#            ('syntactic', 'n_sents'), ('syntactic', 'mean_sent_len'), ('syntactic', 'std_sent_len'), ('syntactic', 'min_sent_len'), ('syntactic', 'max_sent_len'), ('syntactic', 'SPACE'), ('syntactic', 'NOUN'), ('syntactic', 'X'), ('syntactic', 'PUNCT'), ('syntactic', 'VERB'), ('syntactic', 'ADV'), ('syntactic', 'ADP'), ('syntactic', 'ADJ'), ('syntactic', 'CCONJ'), ('syntactic', 'PRON'),
#            ('syntactic', 'DET'), ('syntactic', 'AUX'), ('syntactic', 'PROPN'), ('syntactic', 'PART'), ('syntactic', 'SCONJ'), ('syntactic', 'INTJ'), ('syntactic', 'NUM'),
#            ('lexical', 'n_words'), ('lexical', 'LTR'), ('lexical', 'MALTR'),
#            ('graph', 'number_of_nodes'), ('graph', 'number_of_edges'), ('graph', 'PE'), ('graph', 'LCC'), ('graph', 'LSC'), ('graph', 'degree_average'), ('graph', 'degree_std'), ('graph', 'L1'), ('graph', 'L2'), ('graph', 'L3')
#           ]

### Group difference for sex (absent)

In [ ]:
a = 0.001
s_t_sex, res_t_sex = display_group_test(combined_data_averaged, clinical_cols, 'sex_nr', stats.ttest_ind, stat_name='t', alpha=a)
s_t_sex

In [ ]:
s_t_sex, res_t_sex = display_group_test(combined_data_averaged, cols_av, 'sex_nr', stats.ttest_ind, stat_name='t', alpha=a)
s_t_sex

### Correlative difference for age, education years, and IQ (absent)

In [ ]:
x, y = display_test(combined_data_averaged, cols_av, 'age', stats.pearsonr, stat_name='r', alpha=a)
y[y['abs_r']> 0.2].sort_values('abs_r', ascending=False)

In [ ]:
x, y = display_test(combined_data_averaged, cols_av, 'Bildungsjahre', stats.pearsonr, stat_name='r', alpha=a)
y[y['abs_r']> 0.3].sort_values('abs_r', ascending=False)

In [ ]:
x, y = display_test(combined_data_averaged, cols_av, 'IQ_kristallin', stats.pearsonr, stat_name='r', alpha=a)
y[y['abs_r']> 0.3].sort_values('abs_r', ascending=False)

In [ ]:
# normalized_combined_data_averaged = combined_data_averaged.copy()
# normalized_combined_data_averaged[cols_av] = (combined_data_averaged[cols_av]-combined_data_averaged[cols_av].min())/(combined_data_averaged[cols_av].max()-combined_data_averaged[cols_av].min())

In [ ]:
combined_data_averaged.to_csv('averaged.csv')

# Analyze the metric performance

## Group difference & correlation with PANSS, SANS, and SAPS

In [ ]:
a = 0.001
s_t, res_t = display_group_test(combined_data_averaged, cols_av, 'group', stats.ttest_ind, stat_name='t', alpha=a)

In [ ]:
def corr_test_cols(data, cols_av, traget_cols, corrtest=stats.pearsonr, stat_name='r', alpha=a):
    df = pd.DataFrame()
    for target_col in traget_cols:
        r = display_test(data, cols_av, target_col, stats.pearsonr, stat_name=target_col, alpha=a)[1][target_col]
        df = pd.concat([df, r], axis=1)
    return df

In [ ]:
scales = ["saps_total", "sans_total", "panss", "panss_pos", "panss_neg", "panss_o"]
s_saps, r_saps = display_test(combined_data_averaged, cols_av, 'saps_total', stats.pearsonr, stat_name='r', alpha=a)
s_sans, r_sans = display_test(combined_data_averaged, cols_av, 'sans_total', stats.pearsonr, stat_name='r', alpha=a)
s_panss, r_panss = display_test(combined_data_averaged, cols_av, 'panss_total', stats.pearsonr, stat_name='r', alpha=a)
s_panss_pos, r_panss_pos = display_test(combined_data_averaged, cols_av, 'panss_pos', stats.pearsonr, stat_name='r', alpha=a)
s_panss_neg, r_panss_neg = display_test(combined_data_averaged, cols_av, 'panss_neg', stats.pearsonr, stat_name='r', alpha=a)
s_panss_o, r_panss_o = display_test(combined_data_averaged, cols_av, 'panss_o', stats.pearsonr, stat_name='r', alpha=a)
combined_corr = pd.concat([res_t, r_saps, r_sans, r_panss, r_panss_pos, r_panss_neg, r_panss_o], 
                     keys= ['t_test'] + scales, 
                     names=["scale"], axis=1)

In [ ]:
combined_corr[('mean', 'abs_r')] = combined_corr[[(scale, 'abs_r') for scale in scales]].mean(axis=1)
mean_abs_r = combined_corr.pop(('mean', 'abs_r'))
combined_corr.insert(0, ('mean', 'abs_r'), mean_abs_r) 

In [ ]:
# combined_corr[[(scale, 'abs_r') for scale in ["saps34", "sans12", "panss", "panss_pos", "panss_neg", "panss_o"]]].mean(axis=1)
t_p_n_rank = pd.concat([combined_corr[('t_test', 'abs_t')].rank(ascending=False)] + [combined_corr[(scale, 'abs_r')].rank(ascending=False) for scale in ['sans_total', 'saps_total']], axis=1).mean(axis=1)
mean_rank = pd.concat([combined_corr[(scale, 'abs_r')].rank(ascending=False) for scale in scales], axis=1).mean(axis=1)
combined_corr.insert(1, ('mean', 'rank'), mean_rank) 
combined_corr.insert(2, ('mean', 'rank_abs'), mean_rank.rank()) 
combined_corr.insert(1, ('mean', 'tpn_rank'), t_p_n_rank) 
combined_corr.insert(2, ('mean', 'tpn_rank_abs'), t_p_n_rank.rank()) 

In [ ]:
combined_corr.sort_values(('mean', 'rank'))[['mean', 't_test']].apply(pd.to_numeric).style.background_gradient(axis=0, cmap='Reds')

## Analyze the relation between t-test and correlation with psychiatric scales

In [ ]:
stats.pearsonr(combined_corr[('mean', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
stats.pearsonr(combined_corr[('panss', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
stats.pearsonr(combined_corr[('saps_total', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
stats.pearsonr(combined_corr[('panss_pos', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
stats.pearsonr(combined_corr[('sans_total', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
sns.scatterplot(x=combined_corr[('sans_total', 'abs_r')], y=combined_corr[('t_test', 'abs_t')]);

In [ ]:
stats.pearsonr(combined_corr[('panss_neg', 'abs_r')], combined_corr[('t_test', 'abs_t')])

In [ ]:
sns.scatterplot(x=combined_corr[('panss_neg', 'abs_r')], y=combined_corr[('t_test', 'abs_t')]);

In [ ]:
# combined_corr.sort_values(('mean', 'tpn_rank_abs')).apply(pd.to_numeric).style.background_gradient(axis=0, cmap='Reds')

In [ ]:
# combined_corr.sort_values(('t_test', 'abs_t'), ascending=False).apply(pd.to_numeric).style.background_gradient(axis=0, cmap='Reds')

In [ ]:
# combined_corr.sort_values([('mean', 'abs_r')], ascending=False)

In [ ]:
# combined_corr['t_test', 'abs_t']
# combined_corr['saps_total', 'abs_r']
# combined_corr['sans_total', 'abs_r']
# combined_corr['mean', 'tpn_rank']

In [ ]:
# combined_corr['t_test', 'abs_t']

### Analyze interaction with length

In [ ]:
corr_len_s, corr_len = display_test(combined_data_averaged, cols_av, ('syntactic', 'mean_sent_len'), stats.pearsonr, stat_name='r', alpha=a)

In [ ]:
y = corr_len['abs_r']
tt_norm = (combined_corr['t_test', 'abs_t'] - combined_corr['t_test', 'abs_t'].min()) / (combined_corr['t_test', 'abs_t'].max() - combined_corr['t_test', 'abs_t'].min())

In [ ]:
y = corr_len['abs_r']
z = tt_norm

n = [x[-1] for x in tt_norm.index]
c_map = {'LM': 0, 'graph': 1, 'lexical': 2, 'syntactic': 3}
c = [c_map[label[0]] for label in tt_norm.index]

plt.rcParams["figure.figsize"] = (15,8)
scatter_annotate(z, y, n, 'abs_t norm', 'abs_r len', 'TTEST / LEN', c=c)

In [ ]:
y = corr_len['abs_r']
z = combined_corr['sans_total', 'abs_r']

scatter_annotate(z, y, n, 'abs_r sans', 'abs_r len', 'SANS / LEN', c)

In [ ]:
y = corr_len['abs_r']
z = combined_corr['saps_total', 'abs_r']

scatter_annotate(z, y, n, 'abs_r saps', 'abs_r len', 'SAPS / LEN', c)

In [ ]:
# df = pd.concat([y, combined_corr['t_test', 'abs_t']], axis=1)
# df.columns = ['abs_len_corr', 'abs_ttest']

# fig = px.scatter(df, x='abs_ttest', y='abs_len_corr', text=[x[-1] for x in df.index], log_x=True, size_max=100)
# fig.update_traces(textposition='top center')
# fig.update_layout(title_text='Correlation Length Power vs T-Test Power', title_x=0.5)
# fig.show()

## Analyze correlation tables

In [ ]:
corr = show_corrtest_mask_corr(combined_data_averaged[cols])
# 1. age does not correlate with IQ, education years, or psychiatric scales
# 2. IQ does not correlate with education years or psychiatric scales
# 3. saps-total correlates with saps34 and panss_pos
# 4. sans-total correlates with sans12 and panss_neg, panss_o, panss_total
# 5. panss-total correlates with panss_o, panss_neg, sans-total, panss_pos, saps-total

In [ ]:
combined_data_averaged[['saps34',
                         'saps_total',
                         'sans_total',
                         'sans12',
                         'panss_pos',
                         'panss_neg',
                         'panss_o',
                         'panss_total']].describe()

In [ ]:
corr = show_corrtest_mask_corr(combined_data_averaged[['saps34',
                                                         'saps_total',
                                                         'sans_total',
                                                         'sans12',
                                                         'panss_pos',
                                                         'panss_neg',
                                                         'panss_o',
                                                         'panss_total']])

In [ ]:
corr = show_corrtest_mask_corr(combined_data_averaged[cols_LM + [('syntactic', 'mean_sent_len'), ('syntactic', 'min_sent_len'), ('syntactic', 'n_sents'), ('lexical', 'n_words')]])
# 1. m_lcoh, m_gcoh, m_cgcoh, m_scoh
# 2. m_sprob, m_bert_lcoh, m_bert_gcoh, m_bert_cgcoh, m_bert_scoh
# 3. no corr with mean_sent_len or n_words, m_bert_cgcoh + min_sent_len
# 4. no corr with n_sents, m_cgcoh - n_sents

In [ ]:
corr = show_corrtest_mask_corr(combined_data_averaged[cols_synt])

In [ ]:
corr = show_corrtest_mask_corr(combined_data_averaged[cols_av])

# Control for the effect of length

In [ ]:
compute_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'PUNCT'),
                                  column_names=['punct'], target_name='sans', add_sq=False)[0]

In [ ]:
compute_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'),
                                  column_names=['sent len'], target_name='sans', add_sq=False)[0]

In [ ]:
compute_ortogonolized_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('syntactic', 'PUNCT'),
                                  column_names=['sent len', 'punct'], target_name='sans', add_sq=False)[0]

### bootstrap 

In [ ]:
## 1 sample with replacement

## 1.1 compute_ortogonolized_r for each scale


In [ ]:
scale_cols = ['saps_total',
             'sans_total',
             'panss_pos',
             'panss_neg',
             'panss_o',
             'panss_total']

In [ ]:
def draw_sample_with_replacement(df, seed=None):
    if seed:
        random.seed(seed)
    length = len(df.index)
    idxs = choices(range(length), k=length)
    return df.iloc[idxs, :]

In [ ]:
def t_test(df, column, target_column, test=stats.ttest_ind, nan_policy='omit', stat_name='x', alpha=0.05,
           group_names=None):
    if group_names is None:
        group_names = df[target_column].dropna().unique().tolist()
        assert len(group_names) == 2, 'only two group tests are supported'

    r, p = test(df[df[target_column] == group_names[1]][column],
                df[df[target_column] == group_names[0]][column], nan_policy=nan_policy)
    return r

In [ ]:
t_test(combined_data_averaged, ('syntactic', 'mean_sent_len'), 'group', test=stats.ttest_ind, nan_policy='omit', stat_name='x', alpha=0.05)

In [ ]:
scales = ["saps_total", "sans_total", "panss_total", "panss_pos", "panss_neg", "panss_o"]

def bootstrap(df, cols_av, scale_cols, N, col_to_correct_for=('syntactic', 'mean_sent_len'), group=None):
    dict_scales_sapmles = {k: {scale: {metric: [] for metric in cols_av} for scale in scale_cols} \
                           for k in ('sample_corr', 'sample_raw', 'r', 't', 'r_control')}
    for i in tqdm(range(N)):
        sample = draw_sample_with_replacement(df, seed=i)
        for scale in scale_cols:
            for col in cols_av:
                if group:
                    t_test_res = t_test(sample, col, group)
                    dict_scales_sapmles['t'][scale][col].append(t_test_res)
                    
                r_raw = compute_coefficient(sample, scale, col)[0]
                dict_scales_sapmles['sample_raw'][scale][col].append(r_raw)
                
                droped = sample.dropna(subset=[col, scale])
                r = stats.pearsonr(droped[col], droped[scale])
                dict_scales_sapmles['r'][scale][col].append(r)
                
                if col != col_to_correct_for:
                    
                    droped_c = sample.dropna(subset=[col, col_to_correct_for])
                    r_c = stats.pearsonr(droped_c[col], droped_c[col_to_correct_for])
                    dict_scales_sapmles['r_control'][scale][col].append(r_c)
                    
                    r_corr = compute_ortogonolized_coefficient(sample, scale, col_to_correct_for, col)[0]
                    dict_scales_sapmles['sample_corr'][scale][col].append(r_corr)

    return dict_scales_sapmles

In [ ]:
dict_scales_sapmles = bootstrap(combined_data_averaged, cols_av, scales, 1000, col_to_correct_for=('syntactic', 'mean_sent_len'), group='group')

In [ ]:
reform = {}
for scale in scale_cols:
    for measure in dict_scales_sapmles:
        reform[(scale, measure)] = dict_scales_sapmles[measure][scale]

In [ ]:
reform = {(scale, measure): dict_scales_sapmles[measure][scale] for scale in scale_cols for measure in dict_scales_sapmles}

In [ ]:
median_bootstrap = pd.DataFrame(reform).applymap(np.nanmedian)

In [ ]:
median_bootstrap.loc['LM', 'sans_total']

In [ ]:
sample_df = pd.DataFrame()
for scale in scale_cols:
    scale_metric_sd = {}
    scale_metric_md = {}
    scale_metric_mn = {}
    scale_metric_cov = {}
    scale_metric_diff = {}
    scale_metric_q25 = {}
    scale_metric_q75 = {}
    for col in cols_av:
        if col != ('syntactic', 'mean_sent_len'):
            sample = dict_scales_sapmles['sample_corr'][scale][col]
            scale_metric_sd[col] = np.std(sample)
            scale_metric_md[col] = np.median(sample)
            scale_metric_mn[col] = np.mean(sample)
            scale_metric_q25[col] = np.quantile(sample, 0.25)
            scale_metric_q75[col] = np.quantile(sample, 0.75)
            scale_metric_cov[col] = np.std(sample) / np.mean(sample)
            scale_metric_diff[col] = np.mean(sample) / np.mean(dict_scales_sapmles['sample_raw'][scale][col])
            
    sample_df[(scale, 'md')] = pd.Series(scale_metric_md)
    sample_df[(scale, 'mn')] = pd.Series(scale_metric_mn)
    sample_df[(scale, 'sd')] = pd.Series(scale_metric_sd)
    sample_df[(scale, 'cov')] = pd.Series(scale_metric_cov)
    sample_df[(scale, 'dif')] = pd.Series(scale_metric_diff)
    sample_df[(scale, 'q25')] = pd.Series(scale_metric_q25)
    sample_df[(scale, 'q75')] = pd.Series(scale_metric_q75)

In [ ]:
# sample_df

In [ ]:
sample_df.columns = pd.MultiIndex.from_tuples(sample_df.columns, names=('scale', 'res'))

In [ ]:
ex_df = sample_df.loc['LM', 'sans_total'].drop(index='m_sporb')
ex_df

In [ ]:
ex_df['model'] = ['bert' if 'bert' in x else 'w2v' for x in ex_df.index]
ex_df['metric'] = [x.split('_')[-1] for x in ex_df.index]

In [ ]:
sns.set_theme(style="whitegrid")

# Draw a pointplot to show pulse as a function of three categorical factors
g = sns.catplot(
    data=ex_df, x="model", y="md", hue="metric",
    capsize=.2,  # errorbar="se",
    kind="point", height=6, aspect=.75,
)
g.despine(left=True)

In [ ]:
sample_df.columns = pd.MultiIndex.from_tuples(sample_df.columns, names=('scale', 'res'))
styler = pd.io.formats.style.Styler(sample_df, precision = 4)
styler.background_gradient(axis=0, cmap='Reds')

In [ ]:
sns.histplot(dict_scales_sapmles['sans_total'][('syntactic', 'AUX')], bins=30);

In [ ]:
dict_scales_sapmles

In [ ]:
sns.histplot(dict_scales_sapmles['sans_total'][('LM', 'm_cgcoh')], bins=30);

In [ ]:
sns.histplot(dict_scales_sapmles['sans_total'][('LM', 'm_bert_cgcoh')], bins=30);

### original

In [ ]:
plt.scatter(combined_data_averaged[('syntactic', 'mean_sent_len')], combined_data_averaged[('syntactic', 'AUX')]);

In [ ]:
ex_df = combined_data_averaged[combined_data_averaged[('syntactic', 'mean_sent_len')] > 5]
draw_corrected_scatter(ex_df, 'sans_total', ('syntactic', 'mean_sent_len'), ('syntactic', 'AUX'),
                      column_names=['sent len', 'punct'], target_name='sans')

In [ ]:
compute_ortogonolized_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('syntactic', 'AUX'),
                                  column_names=['sent len', 'punct'], target_name='sans', add_sq=False)[0]

In [ ]:
compute_ortogonolized_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('LM', 'm_cgcoh'),
                                  column_names=['sent len', 'punct'], target_name='sans', add_sq=False)[0]

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('LM', 'm_cgcoh'),
                      column_names=['sent len', 'm_cgcoh'], target_name='sans')

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('syntactic', 'NOUN'),
                      column_names=['sent len', 'num'], target_name='sans')

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('graph', 'number_of_edges'),
                      column_names=['sent len', 'NE'], target_name='sans')

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('graph', 'number_of_nodes'),
                      column_names=['sent len', 'NN'], target_name='sans')

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('lexical', 'MALTR'),
                      column_names=['sent len', 'maltr'], target_name='sans')

In [ ]:
draw_corrected_scatter(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), ('LM', 'm_cgcoh'),
                      column_names=['sent len', 'm_cgcoh'], target_name='sans')

In [ ]:
cols_dict = {}
for col in cols_av:
    cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), col)[0]
combined_corr.insert(1, ('sans_total', 'r_corrected_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
# cols_dict = {}
# for col in cols_av:
#     cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'sans_total', ('syntactic', 'mean_sent_len'), col, add_sq=True)[0]
# combined_corr.insert(1, ('sans_total', 'r_corrected_sq_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
cols_dict = {}
for col in cols_av:
    cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'saps_total', ('syntactic', 'mean_sent_len'), col)[0]
combined_corr.insert(1, ('saps_total', 'r_corrected_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
# cols_dict = {}
# for col in cols_av:
#     cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'saps_total', ('syntactic', 'mean_sent_len'), col, add_sq=True)[0]
# combined_corr.insert(1, ('saps_total', 'r_corrected_sq_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
cols_dict = {}
for col in cols_av:
    cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'panss_total', ('syntactic', 'mean_sent_len'), col)[0]
combined_corr.insert(1, ('panss', 'r_corrected_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
# cols_dict = {}
# for col in cols_av:
#     cols_dict[col] = compute_ortogonolized_coefficient(combined_data_averaged, 'panss_total', ('syntactic', 'mean_sent_len'), col, add_sq=True)[0]
# combined_corr.insert(1, ('panss', 'r_corrected_sq_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
cols_dict = {}
for col in cols_av:
    cols_dict[col] = compute_ortogonolized_logit(combined_data_averaged, 'group', ('syntactic', 'mean_sent_len'), col)
combined_corr.insert(1, ('t_test', 'r_corrected_for_mean_len'), pd.Series(cols_dict)) 

In [ ]:
t_p_n_corr_rank = pd.concat([combined_corr[('t_test', 'abs_t')].rank(ascending=False)] + [combined_corr[(scale, 'r_corrected_for_mean_len')].rank(ascending=False) for scale in ['sans_total', 'saps_total']], axis=1).mean(axis=1)
combined_corr.insert(1, ('mean', 'tpn_rank_corr'), t_p_n_corr_rank) 
combined_corr.insert(2, ('mean', 'tpn_rank_corr_abs'), t_p_n_corr_rank.rank()) 

In [ ]:
combined_corr.sort_values(('mean', 'tpn_rank_corr'), ascending=True).apply(pd.to_numeric).style.background_gradient(axis=0, cmap='Reds')

In [ ]:
def get_ranking_difference(df, col_initial, col_final):
    return df[col_final].rank() - df[col_initial].rank()

In [ ]:
rank_diff = pd.DataFrame()
rank_diff[('panss', 'rank_change')] = get_ranking_difference(combined_corr, ('panss', 'abs_r'), ('panss', 'r_corrected_for_mean_len'))
rank_diff[('panss', 'corrected_rank')] = combined_corr[('panss', 'r_corrected_for_mean_len')].rank()
rank_diff[('panss', 'corrected_r')] = combined_corr[('panss', 'r_corrected_for_mean_len')]


rank_diff[('sans', 'rank_change')] = get_ranking_difference(combined_corr, ('sans_total', 'abs_r'), ('sans_total', 'r_corrected_for_mean_len'))
rank_diff[('sans', 'corrected_rank')] = combined_corr[('sans_total', 'r_corrected_for_mean_len')].rank()
rank_diff[('sans', 'corrected_r')] = combined_corr[('sans_total', 'r_corrected_for_mean_len')]

rank_diff[('saps', 'rank_change')] = get_ranking_difference(combined_corr, ('saps_total', 'abs_r'), ('saps_total', 'r_corrected_for_mean_len'))
rank_diff[('saps', 'corrected_rank')] = combined_corr[('saps_total', 'r_corrected_for_mean_len')].rank()
rank_diff[('saps', 'corrected_r')] = combined_corr[('saps_total', 'r_corrected_for_mean_len')]

rank_diff[('t_test', 'rank_change')] = get_ranking_difference(combined_corr, ('t_test', 'abs_t'), ('t_test', 'r_corrected_for_mean_len'))
rank_diff[('t_test', 'corrected_rank')] = combined_corr[('t_test', 'r_corrected_for_mean_len')].rank()
rank_diff[('t_test', 'corrected_r')] = combined_corr[('t_test', 'r_corrected_for_mean_len')]
rank_diff[('t_test', 'val')] = combined_corr[('t_test', 'abs_t')]

rank_diff.columns = pd.MultiIndex.from_tuples(rank_diff.columns, names=('scale', 'res'))
rank_diff = rank_diff.apply(pd.to_numeric)

In [ ]:
rank_diff.style.background_gradient(axis=0, cmap='Reds')

In [ ]:
n = [label[1] for label in rank_diff.index]
c_map = {'LM': 0, 'graph': 1, 'lexical': 2, 'syntactic': 3}
c = [c_map[label[0]] for label in rank_diff.index]

In [ ]:
def plot_res_corr(x, y, n, x_name, y_name, title, c=None):
    r = stats.pearsonr(x, y).statistic
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    ypred = model.predict(X)
    plt.plot(x, ypred, color='red', label=f'coef: {model.params[1]:.2f} \ncorr: {r:.2f}')

    plt.gca().add_artist(plt.legend(loc=9))

    scatter_annotate(x, y, n, x_name, y_name, title, c=c)

In [ ]:
x = rank_diff[('panss', 'corrected_r')]
y = rank_diff[('t_test', 'corrected_r')]

plot_res_corr(x, y, n, 'panss r', 'ttest pseudo r', 'PANSS / TTEST', c)

In [ ]:
x = rank_diff[('sans', 'corrected_r')]
y = rank_diff[('t_test', 'corrected_r')]

plot_res_corr(x, y, n, 'sans r', 'ttest pseudo r', 'SANS / TTEST', c)

In [ ]:
x = rank_diff[('saps', 'corrected_r')]
y = rank_diff[('t_test', 'corrected_r')]

plot_res_corr(x, y, n, 'saps r', 'ttest pseudo r', 'SAPS / TTEST', c)

In [ ]:
x = rank_diff[('saps', 'corrected_r')]
y = rank_diff[('panss', 'corrected_r')]

plot_res_corr(x, y, n, 'saps r', 'panss r', 'SAPS / PANSS', c)

In [ ]:
x = rank_diff[('sans', 'corrected_r')]
y = rank_diff[('saps', 'corrected_r')]

plot_res_corr(x, y, n, 'sans r', 'saps r', 'SANS / SAPS', c)

In [ ]:
x = rank_diff[('sans', 'corrected_r')]
y = rank_diff[('panss', 'corrected_r')]

plot_res_corr(x, y, n, 'sans r', 'panss r', 'SANS / PANSS', c)

In [ ]:
x = rank_diff[('sans', 'corrected_rank')]
y = rank_diff[('panss', 'corrected_rank')]

plot_res_corr(x, y, n, 'sans rank', 'panss rank', 'SANS / PANSS', c)